In [1]:
import pyro
import pyro.distributions as dist
from pyro.infer.mcmc import MCMC, HMC, NUTS
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import torch.distributions.constraints as constraints

import numpy as np
import scipy.stats as stats
import pandas as pd

import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

%matplotlib inline

In [2]:
assert pyro.__version__.startswith('1.8.1')
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)
# Enable smoke test - run the notebook cells on CI.
#smoke_test = 'CI' in os.environ

In [3]:
nn_decoder = nn.Sequential(nn.Linear(20, 100), nn.Softplus(), nn.Linear(100, 784), nn.Sigmoid())

In [4]:
device = torch.device('cpu')
# if torch.cuda.is_available():
#     device = torch.device('cuda:0')

In [ ]:
import torch.nn.functional as F
# define the PyTorch module that parameterizes the
# diagonal gaussian distribution q(z|x)
class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, input_dim):
        super(Encoder, self).__init__()
        self.z_dim = z_dim
        self.input_dim = input_dim
        # setup the three linear transformations used
        self.fc1 = nn.Linear(input_dim[0]*input_dim[1], hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim**2)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, x):
        # then compute the hidden units
        hidden = self.softplus(self.fc1(x.reshape(x.shape[0], self.input_dim[0] * self.input_dim[1])))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc21(hidden)
        A_vect = self.fc22(hidden)
        A = A_vect.reshape((A_vect.shape[0], self.z_dim, self.z_dim))
        A = torch.tril(A, diagonal=0)
        for idx in range(5):
            A[:,idx, idx] = torch.exp(A[:,idx, idx])
        try:
            torch.cholesky(z_scale)
            z_scale = torch.bmm(A, A.transpose(1, 2))
            return z_loc, z_scale
        except:
            for idx in range(A.shape[0]):
                try:
                    z_scale = torch.bmm(A[idx,:,:], A[idx,:,:].transpose(1, 2))
                    torch.cholesky(z_scale)
                except:
                    laplace = 1e-9
                    while True:
                        for v_idx in range(5):
                            for h_idx in range(5):
                                if v_idx < h_idx:
                                    continue
                                try:
                                    A_c = A[idx,:,:]
                                    A_c[v_idx, h_idx] += 1e-9
                                    z_scale = torch.bmm(A_c, A_c.transpose(1, 2))
                                    torch.cholesky(z_scale)
                                    A[idx,:,:] = A_c
                                    break
                                except:
                                    laplace += 1
                                    continue
                            break
                        break
        z_scale = torch.bmm(A, A.transpose(1, 2))              
        return z_loc, z_scale


# define the PyTorch module that parameterizes the
# observation likelihood p(x|z)
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, input_dim):
        super(Decoder, self).__init__()
        # setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, input_dim)
        self.fc22 = nn.Linear(hidden_dim, input_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        mu = self.fc21(hidden)
        sigma = torch.exp(self.fc22(hidden))
        return mu, sigma


# define a PyTorch module for the VAE
class VAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, input_dim,
        z_dim=5, hidden_dim=250, use_cuda=False):
        super(VAE, self).__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim, input_dim=input_dim)
        # self.decoder = Decoder(z_dim, hidden_dim, input_dim=input_dim)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
        self.use_cuda = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    def model(self, sigma, x):
        # register PyTorch module `decoder` with Pyro
        # pyro.module("decoder", self.decoder)
        with pyro.plate("data", x.shape[0]):
            # setup hyperparameters for prior p(z)
            mu_loc = torch.zeros(x.shape[0], self.z_dim, dtype=x.dtype, device=x.device)
            mu_scale = torch.eye(x[0].shape[1]).reshape(
                (1,x[0].shape[1],x[0].shape[1])).repeat(x.shape[0], 1, 1)
            # sample from prior (value will be sampled by guide when computing the ELBO
            # print("Dimension of mu_scale {}".format(mu_scale.shape))
            # print("Dimension of mu_loc {}".format(mu_loc.shape))
            mu = pyro.sample("latent", dist.MultivariateNormal(mu_loc, mu_scale))
            # print("Dimension of mu {}".format(mu.shape))
            # score against actual images
            # print(x.shape)
            pyro.sample("obs", dist.MultivariateNormal(mu, sigma), obs=x.mean())
            # print("Here")
    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, sigma, x):
        # register PyTorch module `encoder` with Pyro
        pyro.module("encoder", self.encoder)
        with pyro.plate("data", x.shape[0]):
            # use the encoder to get the parameters used to define q(z|x)
            z_loc, z_scale = self.encoder.forward(x)
            # print(z_loc.shape)
            # print(z_scale.shape)
            # sample the latent code z
            pyro.sample("latent", dist.MultivariateNormal(z_loc, z_scale))

In [6]:
X = []
for idx in range(100):
    random_means = np.random.sample(size=5) 
    
    x_i = multivar_random(5, random_means, [1,1,1,1], num_samples=500)[0]
    X.append(x_i)

In [23]:
F.log_softmax(x_i)

C:\Users\kylea\Miniconda3\envs\devenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[-2.6515e+00, -3.1009e+00, -2.0017e+00, -1.4382e+01, -2.8855e-01],
        [-3.9868e-01, -1.7197e+00, -1.9020e+00, -7.9675e+00, -9.8578e+00],
        [-4.3851e-01, -5.0679e+00, -2.9041e+00, -8.5794e+00, -1.2251e+00],
        ...,
        [-5.3434e-01, -2.9726e+00, -1.0164e+00, -1.1418e+01, -7.0515e+00],
        [-2.5917e+00, -8.9148e-01, -2.2809e+00, -1.6618e+01, -8.8463e-01],
        [-1.4421e+01, -1.8586e+01, -1.8143e+01, -2.5163e+01, -5.9605e-07]])

In [29]:
mu = torch.Tensor([0] * 100)
sd = torch.Tensor([1] * 100)

p = torch.distributions.Normal(mu,sd)
q = torch.distributions.Normal(mu,sd)

out = torch.distributions.kl_divergence(p, q)

In [31]:
q

Normal(loc: torch.Size([100]), scale: torch.Size([100]))

In [22]:
from scipy.stats import norm
np.log(norm.pdf(x=1.846, loc=0.4659, scale=1))

-1.8712765382046728

In [ ]:
# clear param store
pyro.clear_param_store()

no_instances = 20000
input_dim = (500, 5)
mu = stats.norm.rvs(size=input_dim)

# setup the VAE
vae = VAE(use_cuda=False, input_dim=input_dim, z_dim=5)

adam_args = {"lr": 0.001}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())
train_loader = DataLoader(X, batch_size=50, shuffle=True,
     num_workers=1, pin_memory=True, drop_last=False)

train_elbo = []
for epoch in range(100):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x in train_loader:
        # x = x.cuda()
        epoch_loss += svi.step(torch.eye(5), x)

    # report training diagnostics
    if not epoch % 10:
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo.append(total_epoch_loss_train)
        print("[epoch %03d]  average training loss: %.4f" %
             (epoch, total_epoch_loss_train))

In [200]:
L = np.array([[ 3.6956e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.0094e-01,  1.0461e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 3.1744e+00,  2.9765e+00,  2.6687e-02,  0.0000e+00,  0.0000e+00],
        [-6.0710e+00,  4.9560e-03,  2.6047e+00,  1.4794e-01,  0.0000e+00],
        [ 1.4442e+00, -1.7773e+00,  4.8894e-01, -1.9205e+00,  2.4491e-05]])
for v_idx in range(5):
    for h_idx in range(5):
        if v_idx < h_idx:
            continue
        try:
            L_c = L.copy()
            L_c[v_idx, h_idx] += 1e-9
            np.linalg.cholesky(np.dot(L_c, L_c.T))
            print("Success, removed {}".format(L[v_idx, h_idx]))
        except:
            None

Success, removed 0.0036956
Success, removed 3.1744
Success, removed 2.9765
Success, removed 0.026687


In [192]:
pd.DataFrame(np.dot(L, L.T))

,0,1,2,3,4
0,0.000014,0.000373,0.011731,-0.022436,0.005337
1,0.000373,109.442710,31.457590,-0.560962,-18.446558
2,0.011731,31.457590,18.937080,-19.187519,-0.692617
3,-0.022436,-0.560962,-19.187519,43.663414,-7.787123
4,0.005337,-18.446558,-0.692617,-7.787123,9.171892


In [152]:
np.linalg.cholesky([[ 1.3657e-05,  3.7304e-04,  1.1731e-02, -2.2436e-02,  5.3372e-03],
        [ 3.7304e-04,  1.0943e+02,  3.1456e+01, -5.6098e-01, -1.8445e+01],
        [ 1.1731e-02,  3.1456e+01,  1.8937e+01, -1.9187e+01, -6.9259e-01],
        [-2.2436e-02, -5.6098e-01, -1.9187e+01,  4.3663e+01, -7.7871e+00],
        [ 5.3372e-03, -1.8445e+01, -6.9259e-01, -7.7871e+00,  9.1720e+00]])

LinAlgError: Matrix is not positive definite

In [ ]:
mean = torch.Tensor([[1, 2, 3], [4, 5, 6]])
cov1 = torch.eye(3)
cov2 = torch.Tensor([[1, 1, 1], [1, 2, 2], [1, 2, 3]])
cov = torch.stack([cov1, cov2], 0)
distrib = dist.MultivariateNormal(loc=mean, covariance_matrix=cov)
distrib.sample()

In [5]:
def multivar_random(size, normal_params, gamma_params, num_samples):   

    # simulate two random covariance matrices (PSD) for sigma and sigma_0
    mat = np.random.gamma(gamma_params[0], gamma_params[1], size=size**2).reshape((size, size))
    sigma_mat = torch.tensor(np.dot(mat, mat.transpose()))
    
    mat_0 = np.random.gamma(gamma_params[2], gamma_params[3], size=size**2).reshape((size, size))
    sigma_mat_0 = torch.tensor(np.dot(mat_0, mat_0.transpose()))
    
    # simulate some random means
    mu_vector = torch.tensor(np.random.multivariate_normal(normal_params, sigma_mat_0, size=1))[0]

    # now simulate multivariate gaussian
    data = torch.tensor(np.random.multivariate_normal(mu_vector, sigma_mat, size=num_samples).astype("float32"))
    
    return(data, mu_vector, sigma_mat, sigma_mat_0)

In [ ]:
train_loader

In [ ]:
# clear param store
pyro.clear_param_store()

no_instances = 20000
input_dim = 2
mu = stats.norm.rvs(size=input_dim)

# Generate a positive definite matrix
sigma = stats.norm.rvs(size=(input_dim, input_dim))
sigma[np.triu_indices(input_dim)] = 0
sigma += np.diag(np.abs(stats.norm.rvs(size=input_dim)))
sigma = np.matmul(sigma.transpose(), sigma) # inverse cholesky decomposition

dataset = stats.multivariate_normal.rvs(mu, sigma, size=no_instances)
dataset = torch.as_tensor(dataset, dtype=torch.float32)
dataset = TensorDataset(dataset)
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True,
     num_workers=1, pin_memory=True, drop_last=False)

# setup the VAE
vae = VAE(use_cuda=False, input_dim=input_dim, z_dim=2)

adam_args = {"lr": 0.001}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
for epoch in range(100):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, in train_loader:
        #x = x.cuda()
        epoch_loss += svi.step(x)

    # report training diagnostics
    if not epoch % 10:
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo.append(total_epoch_loss_train)
        print("[epoch %03d]  average training loss: %.4f" %
             (epoch, total_epoch_loss_train))

In [70]:
test = stats.multivariate_normal.rvs(mu, sigma, size=no_instances)
test = torch.as_tensor(test, dtype=torch.float32)
vae.encoder(test)[0]

NameError: name 'sigma' is not defined

In [71]:
no_instances

20000

In [72]:
# Generating new instances (replications) from the trained VAE
new_instances = vae.new_instances(100000)

print("True means")
print(mu)
print("Empirical means of replications:")
print(new_instances.mean(0))

print("----------------------------------------")

print("True covariance matrix")
print(sigma)
print("Empirical covariance matrix of replications:")
print(np.cov(new_instances, rowvar=False))

AttributeError: 'VAE' object has no attribute 'new_instances'

In [21]:
# clear param store
pyro.clear_param_store()
loc = 5
scale = 20
data = torch.tensor(np.random.normal(loc=loc, scale=scale, size=1000))
dataset = TensorDataset(data)
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True,
     num_workers=1, pin_memory=True, drop_last=False)

# setup the VAE
vae = VAE(use_cuda=False, input_dim=input_dim)

adam_args = {"lr": 0.001}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
for epoch in range(100):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, in train_loader:
        #x = x.cuda()
        epoch_loss += svi.step(x)

    # report training diagnostics
    if not epoch % 10:
        normalizer_train = len(train_loader.dataset)
        total_epoch_loss_train = epoch_loss / normalizer_train
        train_elbo.append(total_epoch_loss_train)
        print("[epoch %03d]  average training loss: %.4f" %
             (epoch, total_epoch_loss_train))

NameError: name 'loc' is not defined